In [8]:
%load_ext autoreload
%autoreload 2

from models import Detr
from utils import load_config
import torch

config = load_config(r"configs\train.yaml")

model = Detr(
    train=True,
    lr=1e-4,
    lr_backbone=1e-5,
    weight_decay=1e-4,
    pretrained='facebook/detr-resnet-101-dc5',
    checkpoint = None,
    config=config)
model.train()

device = torch.device('cuda:0')
model.to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of the model checkpoint at facebook/detr-resnet-101-dc5 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model chec

Detr(
  (model): DetrForObjectDetection(
    (model): DetrModel(
      (backbone): DetrConvModel(
        (conv_encoder): DetrConvEncoder(
          (model): FeatureListNet(
            (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            (bn1): DetrFrozenBatchNorm2d()
            (act1): ReLU(inplace=True)
            (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
            (layer1): Sequential(
              (0): Bottleneck(
                (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn1): DetrFrozenBatchNorm2d()
                (act1): ReLU(inplace=True)
                (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): DetrFrozenBatchNorm2d()
                (drop_block): Identity()
                (act2): ReLU(inplace=True)
                (aa): Identity()
                (conv3): Conv2d(64, 256

In [9]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

# Set up checkpointing and trainer
checkpoint_callback = ModelCheckpoint(
    monitor="validation/loss",
    filename="detr-{epoch:02d}-{validation/loss:.2f}",
    save_top_k=3,
    mode="min",
    save_last=True,
    dirpath=r"checkpoints"
)

trainer = Trainer(
    max_epochs=50,
    log_every_n_steps=10,
    callbacks=[checkpoint_callback],
    val_check_interval=0.5,  # Run validation 2 times per epoch
    devices=1,
    accelerator="gpu",
    gradient_clip_val=0.1,
    accumulate_grad_batches=48
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                   | Params | Mode 
--------------------------------------------------------------------
0 | model            | DetrForObjectDetection | 60.4 M | train
1 | simplified_tbnet | SimplifiedTbNet        | 263 K  | train
--------------------------------------------------------------------
60.5 M    Trainable params
222 K     Non-trainable params
60.7 M    Total params
242.822   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[{'size': tensor([ 338, 1333]), 'image_id': tensor([38]), 'class_labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'boxes': tensor([[0.0581, 0.0750, 0.0867, 0.0833],
        [0.4228, 0.0750, 0.0550, 0.0833],
        [0.8161, 0.0750, 0.0761, 0.0833],
        [0.3372, 0.1792, 0.0233, 0.0750],
        [0.5074, 0.1792, 0.0761, 0.0750],
        [0.7030, 0.1792, 0.0233, 0.0750],
        [0.8816, 0.1792, 0.0761, 0.0750],
        [0.0624, 0.3083, 0.0951, 0.1000],
        [0.3372, 0.3083, 0.0529, 0.1000],
        [0.5074, 0.3083, 0.1311, 0.1000],
        [0.7030, 0.3083, 0.0529, 0.1000],
        [0.8816, 0.3083, 0.1311, 0.1000],
        [0.0624, 0.4083, 0.0951, 0.1000],
        [0.3372, 0.4083, 0.0529, 0.1000],
        [0.5074, 0.4083, 0.1311, 0.1000],
        [0.7030, 0.4083, 0.0529, 0.1000],
        [0.8816, 0.4083, 0.1311, 0.1000],
        [0.1311, 0.5083, 0.2326, 0.1000],
        [0.3383, 

Training: |          | 0/? [00:00<?, ?it/s]

[{'size': tensor([ 722, 1333]), 'image_id': tensor([5]), 'class_labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'boxes': tensor([[0.0916, 0.0809, 0.1753, 0.0588],
        [0.6693, 0.0809, 0.3187, 0.0588],
        [0.1016, 0.2096, 0.1952, 0.0662],
        [0.7092, 0.2096, 0.3984, 0.0662],
        [0.1235, 0.2904, 0.2390, 0.0662],
        [0.7012, 0.2904, 0.3825, 0.0662],
        [0.0637, 0.3750, 0.1195, 0.0588],
        [0.7072, 0.3750, 0.3944, 0.0588],
        [0.0857, 0.4596, 0.1633, 0.0662],
        [0.7092, 0.4596, 0.3984, 0.0662],
        [0.0896, 0.5404, 0.1713, 0.0662],
        [0.6952, 0.5404, 0.3705, 0.0662],
        [0.1096, 0.6250, 0.2112, 0.0588],
        [0.7012, 0.6250, 0.3825, 0.0588],
        [0.0916, 0.7096, 0.1753, 0.0662],
        [0.6813, 0.7096, 0.3426, 0.0662],
        [0.1135, 0.7941, 0.2191, 0.0588],
        [0.6912, 0.7941, 0.3625, 0.0588],
        [0.0936, 0.8750, 0.1793, 0.0588],
        [0.6992, 0.8750, 0.3785, 0.0588],
  